In [0]:


from aaintdatapipeline.application.app_qa.db_check_v1.test_app_store_rank import check_app_store_rank

check_app_store_rank({"project_name": "All_Products"})


In [0]:

# Copyright (c) 2018 App Annie Inc. All rights reserved.

"""
DB Check modules
"""
import datetime
from collections import defaultdict

from aaintdatapipeline.application.app_qa.common.db_check_utils import query, send_report
from aaintdatapipeline.application.app_qa.conf import settings


APP_STORE_RANK_METRICS = ["free_download", "new_paid_download", "revenue", "paid_download", "new_free_download"]


def citus_settings(schema):
    template = "CITUS_" + schema.upper() + "_CITUS_DB_{property}"
    host, port = getattr(settings, template.format(property='HOSTS'))[0]
    return (
        "dbname='{db}' user='{user}' password='{password}' "
        "host='{host}' port='{port}'".format(
            db=getattr(settings, template.format(property='NAME')),
            user=getattr(settings, template.format(property='ACCESS_ID')),
            host=host,
            password=getattr(settings, template.format(property='SECRET_KEY')),
            port=port
        )
    )

def check_app_store_rank(params):
    project_name = params['project_name']
    if project_name == 'All_Products':
        check_date = datetime.date.today() - datetime.timedelta(days=2)
        app_store_rank_dimension_completeness_check_test(date=check_date)
        app_store_rank_rank_completeness_check_test(date=check_date)


@send_report(source="App Store Rank - Dimension Completeness Check")
def app_store_rank_dimension_completeness_check_test(date):
    device_code_list = ['ios-tablet', 'ios-phone', 'android-all']
    ios_countries_list = ['US', 'AU', 'JP', 'TT', 'VN', 'NE']
    gp_countries_list = ['US', 'AU', 'JP', 'TT', 'VN', 'PY']
    failed_ids = defaultdict(list)
    for device_code in device_code_list:
        if device_code != 'android-all':
            for country_code in ios_countries_list:
                rank_data = check_app_store_rank_date('100001', date, device_code, country_code)
                if not rank_data[0][0]:
                    failed_ids[device_code].append(country_code)
        else:
            for country_code in gp_countries_list:
                rank_data = check_app_store_rank_date('400025', date, device_code, country_code)
                if not rank_data[0][0]:
                    failed_ids[device_code].append(country_code)
    if failed_ids:
        raise RuntimeError('Failed on bellow ids: {}'.format(failed_ids))


def check_app_store_rank_date(category_id, date, device_code, country_code):
    sql = """
    select count(1) from store.store_app_rank_fact_v1 where granularity = 'hourly' and hour = 23 and date = '{date}'
            and device_code = '{device_code}' and country_code = '{country_code}' and category_id = {category_id};
    """.format(
        category_id=category_id, device_code=device_code, date=date, country_code=country_code)
    result = query(citus_settings("aa"), sql)
    return result


@send_report(source="App Store Rank - Rank Completeness Check")
def app_store_rank_rank_completeness_check_test(date):
    for metric in APP_STORE_RANK_METRICS:
        sql = 'SELECT device_code, country_code, category_id, count({metric}),max({metric}) FROM s' \
              'tore.store_app_rank_fact_v1  WHERE date = \'{date_str}\' AND ( granularity = \'hourly\' ) ' \
              'AND ( hour = 23 ) AND ({metric} is not null)  GROUP BY device_code, country_code, category_id'.format(
                  metric=metric, date_str=date)
        results = query(citus_settings("aa"), sql)
        error_logs = []
        for result in results:
            device_code, country_code, category_id, count, max = result
            if count != 0 and count != max:
                log = ",".join([device_code, country_code, str(category_id), metric, str(count),str(max)])
                error_logs.append(log)
    raise RuntimeError(log)

check_app_store_rank({"project_name": "All_Products"})

